## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, use_current_place_point_only, use_current_sub_binders_only, finish_L_shape

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection command:
indy0: False
panda1: False


In [3]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming


[(array([[1.82983423e+03, 0.00000000e+00, 1.91572046e+03],
         [0.00000000e+00, 1.82983423e+03, 1.09876086e+03],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([ 7.09966481e-01, -2.73409390e+00,  1.45804870e-03, -3.24774766e-04,
          1.44911301e+00,  5.84310412e-01, -2.56374550e+00,  1.38472950e+00])),
 (array([[1.39560388e+03, 0.00000000e+00, 9.62751587e+02],
         [0.00000000e+00, 1.39531934e+03, 5.47687012e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  array([0., 0., 0., 0., 0.])),
 array([[ 0.8263327 , -0.02442539, -0.56265235,  0.650057  ],
        [ 0.17226346,  0.9621378 ,  0.21122517, -0.15914251],
        [ 0.53618985, -0.2714667 ,  0.7992536 ,  0.09105545],
        [ 0.        ,  0.        ,  0.        ,  1.        ]],
       dtype=float32)]

## create scene builder

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [5]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [6]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [7]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

   Use a production WSGI server instead.
 * Debug mode: off
Please create a subscriber to the marker


## add indy tool

In [8]:
add_indy_sweep_tool(gscene, "indy0", face_name="brush_face")

## add sweep face

In [9]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [10]:
gscene.update_markers_all()

## add wp

In [11]:
TOOL_DIM = [0.06, 0.10]
TACK_WIDTH = 0.17
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
WP_REF_M = np.array([0, WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    face = gscene.create_safe(GEOTYPE.BOX, "face{}".format(i_trc+1), "base_link", 
                              (TRACK_DIM[0] - TOOL_DIM[0]/2,TACK_WIDTH,TRC_THIC), 
                             center=tuple(WP_REF_M+[0,TRACK_STEP*i_trc])+(1e-2,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2, face))

In [12]:
gscene.update_markers_all()

## add blocking structure on table

In [13]:
# gscene.create_safe(gtype=GEOTYPE.BOX, name="blocker", link_name="base_link",
#                             center=(-0.27,-0.2,0.15), dims=(0.1,0.3, 0.4), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
#                             collision=True, fixed=True)

## add box

In [14]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
finish_L_shape(gscene, gtem_dict)

In [15]:
gscene.update_markers_all()

## Register binders

In [16]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [17]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)

## add objects

In [18]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [19]:
for gname in sorted(gtem_dict.keys()):
    pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [20]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [21]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2, face = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                   action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])}, 
                                   clearance=[face])
    sweep_list.append(sweep_)

### planners

In [22]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [23]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
tcheck = TaskClearanceChecker(pscene, gcheck)
checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [24]:
mplan.motion_filters = checkers_all

In [25]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [26]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)

('track_face', 'track_face', 'track_face', 0, 0, 0)


In [27]:
pscene.subject_name_list

['box1', 'box2', 'box_long', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [28]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [29]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan & show

In [30]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset()

In [31]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
    finish_L_shape(gscene, gtem_dict)

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
gscene.update_markers_all()

# remove place points and sub-binders except for the current ones
use_current_place_point_only(pscene, initial_state)
use_current_sub_binders_only(pscene, initial_state)
tplan.prepare()
mplan.update_gscene()

In [32]:
mplan.reset_log(False)
gtimer.reset()

with gtimer.block("detect"):
    ## detect again
    gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
    finish_L_shape(gscene, gtem_dict)

    ## create boxes
    box_list =  [] 
    for gname in sorted(gtem_dict.keys()):
        box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
        box_new.register_binders(pscene, PlacePlane)
        box_list.append(box_new)
    
    initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
    gscene.update_markers_all()

    # remove place points and sub-binders except for the current ones
    use_current_place_point_only(pscene, initial_state)
    use_current_sub_binders_only(pscene, initial_state)
    tplan.prepare()
    mplan.update_gscene()
    
print(initial_state.node)

gtimer.tic("firstmove")
obj_num = len(gtem_dict)
sweep_num = len(sweep_list)
from_state = initial_state
t_exe = None
snode_schedule_all = []
for sweep_idx in range(sweep_num):
    gcheck.put_banned = [track_list[sweep_idx][2]]
    sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
    goal_nodes = [("track_face",)*obj_num+sweep_goal]
    if sweep_idx < sweep_num-1:
        for i_s in range(obj_num):
            obj_goal = ["track_face"]*obj_num
            obj_goal[i_s] = "grip1"
            goal_nodes += [tuple(obj_goal)+sweep_goal]
    gtimer.tic("plan{}".format(sweep_idx))
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=False, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
    gtimer.toc("plan{}".format(sweep_idx))
    schedules = ppline.tplan.find_schedules()
    schedules_sorted = ppline.tplan.sort_schedule(schedules)
    snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
    snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
    snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
    snode_schedule = mix_schedule(mplan, snode_schedule_safe)
    from_state = snode_schedule[-1].state
    if t_exe:
        t_exe.join()
    else:
        if len(snode_schedule_all)==0:
            gtimer.toc("firstmove")
    snode_schedule_all.append(snode_schedule)
#     t_exe = Thread(target=ppline.play_schedule, args = (snode_schedule,), kwargs=dict( period=0.01))
#     t_exe.start()
# t_exe.join()

('box2_top', 'box_long_right', 'track_face', 0, 0, 0)
try: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('box2_top', 'box_long_right', 'track_face', 1, 0, 0)
result: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('box2_top', 'box_long_right', 'track_face', 1, 0, 0) = fail
try: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('box2_top', 'grip1', 'track_face', 0, 0, 0)
actor_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
object_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 'indy0_link3', 'indy0_link4', 'indy0_link5', 'indy0_link6', 'indy0_tcp']
result: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('box2_top', 'grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0)
actor_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
object_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 

transition motion tried: True
result: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0) = success
branching: 0->3 (0.68/50.0 s, steps/err: 54(244.338989258 ms)/0.00151985087811)
try: 3 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0)
try joint motion
joint motion tried: True
result: 3 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0) = success
branching: 3->4 (0.78/50.0 s, steps/err: 34(106.718063354 ms)/7.68116237724e-16)
try: 3 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 1, 0, 0)
result: 3 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 1, 0, 0) = fail
try: 4 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 1, 0, 0)
result: 4 - ('grip1', 'box_long_right', 'track_face', 0,

joint motion tried: True
result: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'track_face', 0, 0, 0) = success
branching: 5->6 (1.22/50.0 s, steps/err: 47(150.595188141 ms)/7.67194929748e-16)
try: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'track_face', 1, 0, 0)
result: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'track_face', 1, 0, 0) = fail
try: 6 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'track_face', 1, 0, 0)
result: 6 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'track_face', 1, 0, 0) = fail
try: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'grip1', 'track_face', 0, 0, 0)
actor_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
object_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link

try: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'grip1', 'track_face', 0, 0, 0)
result: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'grip1', 'track_face', 0, 0, 0) = fail
try: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'grip1', 'track_face', 0, 0, 0)
actor_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
object_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 'indy0_link3', 'indy0_link4', 'indy0_link5', 'indy0_link6', 'indy0_tcp']
result: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'grip1', 'track_face', 0, 0, 0) = fail
try: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0)
result: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0) = fail
try: 1 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('track_face

result: 5 - ('track_face', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'grip1', 0, 0, 0) = fail
try: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0)
actor_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
object_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 'indy0_link3', 'indy0_link4', 'indy0_link5', 'indy0_link6', 'indy0_tcp']
result: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('grip1', 'box_long_right', 'track_face', 0, 0, 0) = fail
try: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('box2_top', 'box_long_right', 'grip1', 0, 0, 0)
result: 0 - ('box2_top', 'box_long_right', 'track_face', 0, 0, 0)->('box2_top', 'box_long_right', 'grip1', 0, 0, 0) = fail
try: 4 - ('grip1', 'box_long_right', 'track_face', 0, 0, 0)->('track_face', 'box_long_right', 'track_face', 0, 0, 0)
actor_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'ind

...

actor_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 'indy0_link3', 'indy0_link4', 'indy0_link5', 'indy0_link6', 'indy0_tcp']
object_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
result: 0 - ('track_face', 'track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 'track_face', 2, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 'track_face', 2, 0, 0)
actor_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 'indy0_link3', 'indy0_link4', 'indy0_link5', 'indy0_link6', 'indy0_tcp']
object_Tinv_dict: ['panda1_hand', 'panda1_link7', 'panda1_link8']
result: 0 - ('track_face', 'track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 'track_face', 2, 0, 0) = fail
try: 0 - ('track_face', 'track_face', 'grip1', 2, 0, 0)->('track_face', 'track_face', 'track_face', 2, 0, 0)
actor_Tinv_dict: ['base_link', 'indy0_link0', 'indy0_link1', 'indy0_link2', 'indy0_link3', 'indy0_link4', 'indy0_l

result: 1 - ('track_face', 'track_face', 'track_face', 2, 0, 0)->('track_face', 'grip1', 'track_face', 2, 0, 0) = success
branching: 1->7 (6.62/50.0 s, steps/err: 55(257.488012314 ms)/0.00134418006604)
try: 7 - ('track_face', 'grip1', 'track_face', 2, 0, 0)->('track_face', 'grip1', 'track_face', 2, 0, 0)
try joint motion
joint motion tried: True
result: 7 - ('track_face', 'grip1', 'track_face', 2, 0, 0)->('track_face', 'grip1', 'track_face', 2, 0, 0) = success
branching: 7->8 (6.69/50.0 s, steps/err: 36(61.1851215363 ms)/7.697627732e-16)
try: 7 - ('track_face', 'grip1', 'track_face', 2, 0, 0)->('track_face', 'grip1', 'track_face', 2, 1, 0)
result: 7 - ('track_face', 'grip1', 'track_face', 2, 0, 0)->('track_face', 'grip1', 'track_face', 2, 1, 0) = fail
try: 8 - ('track_face', 'grip1', 'track_face', 2, 0, 0)->('track_face', 'grip1', 'track_face', 2, 1, 0)
actor_Tinv_dict: ['indy0_link6', 'indy0_tcp']
object_Tinv_dict: ['base_link', 'panda1_hand', 'panda1_link0', 'panda1_link1', 'panda1_l

transition motion tried: True
result: 0 - ('track_face', 'grip1', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 0) = success
branching: 0->1 (0.39/50.0 s, steps/err: 112(250.219106674 ms)/0.00138663466998)
try: 1 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 0)
try joint motion
joint motion tried: True
result: 1 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 0) = success
branching: 1->2 (0.59/50.0 s, steps/err: 108(194.855213165 ms)/1.00405258355e-15)
generate table - PlanConditions
try: 1 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 1)
generate table - PlanList
result: 1 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 1) = fail
try: 2 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2

transition motion tried: True
result: 1 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 'track_face', 2, 2, 0) = success
branching: 1->5 (7.11/50.0 s, steps/err: 113(384.605169296 ms)/0.00167602800294)
try: 5 - ('grip1', 'track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 'track_face', 2, 2, 0)
try joint motion
joint motion tried: True
result: 5 - ('grip1', 'track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 'track_face', 2, 2, 0) = success
branching: 5->6 (7.18/50.0 s, steps/err: 44(75.4940509796 ms)/1.05193054321e-15)
try: 5 - ('grip1', 'track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 'track_face', 2, 2, 1)
result: 5 - ('grip1', 'track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 'track_face', 2, 2, 1) = fail
try: 6 - ('grip1', 'track_face', 'track_face', 2, 2, 0)->('grip1', 'track_face', 'track_face', 2, 2, 1)
actor_Tinv_dict: ['indy0_link6', 'indy0_tcp']
object_Tinv_dict: ['base_link', 'panda1_hand', 'panda1_

joint motion tried: True
result: 9 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 0) = success
branching: 9->10 (12.93/50.0 s, steps/err: 50(114.259004593 ms)/9.96199465848e-16)
try: 9 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 1)
actor_Tinv_dict: ['indy0_link6', 'indy0_tcp']
object_Tinv_dict: ['base_link', 'panda1_hand', 'panda1_link0', 'panda1_link1', 'panda1_link2', 'panda1_link3', 'panda1_link4', 'panda1_link5', 'panda1_link6', 'panda1_link7', 'panda1_link8']
result: 9 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 1) = fail
try: 10 - ('track_face', 'track_face', 'track_face', 2, 2, 0)->('track_face', 'track_face', 'track_face', 2, 2, 1)
actor_Tinv_dict: ['indy0_link6', 'indy0_tcp']
object_Tinv_dict: ['base_link', 'panda1_hand', 'panda1_link0', 'panda1_link1', 'panda1_link2', 'panda1_link3', 'panda1_link4', 'p

In [33]:
print(gtimer)

detect: 	531.0 ms/1 = 531.128 ms (531.128/531.128)
firstmove: 	6709.0 ms/1 = 6709.085 ms (6709.085/6709.085)
plan0: 	6195.0 ms/1 = 6194.765 ms (6194.765/6194.765)
initialize_memory: 	0.0 ms/3 = 0.105 ms (0.039/0.17)
init_search: 	121.0 ms/3 = 40.32 ms (39.226/42.24)
test_connection: 	25565.0 ms/508 = 50.325 ms (1.414/5019.928)
plan1: 	7191.0 ms/1 = 7190.66 ms (7190.66/7190.66)
plan2: 	13709.0 ms/1 = 13708.811 ms (13708.811/13708.811)



## mix full schedule

In [34]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

mix_schedule_safe: 	158.0 ms/1 = 158.397 ms (158.397/158.397)



### play schedule

In [ ]:
ppline.play_schedule(safe_mixed, period=0.001)

### Run fully mixed schedule

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
ppline.execute_schedule(safe_mixed)

## Plan & execute

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)
time.sleep(1)
crob.grasp(True, True)
time.sleep(1)
crob.grasp(False, False)
crob.joint_move_make_sure(crob.home_pose+0.05)
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
for _ in range(3):
    indy = crob.robot_dict['indy0']
    with indy:
        indy.wait_di(16)

    mplan.reset_log(False)
    gtimer.reset()

    with gtimer.block("detect"):
        ## detect again
        gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
        finish_L_shape(gscene, gtem_dict)

        ## create boxes
        box_list =  [] 
        for gname in sorted(gtem_dict.keys()):
            box_new = pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True)
            box_new.register_binders(pscene, PlacePlane)
            box_list.append(box_new)

        initial_state = pscene.initialize_state(crob.home_pose, force_fit_binding=True)
        gscene.update_markers_all()

        # remove place points and sub-binders except for the current ones
        use_current_place_point_only(pscene, initial_state)
        use_current_sub_binders_only(pscene, initial_state)
        tplan.prepare()
        mplan.update_gscene()

    print(initial_state.node)

    gtimer.tic("firstmove")
    obj_num = len(gtem_dict)
    sweep_num = len(sweep_list)
    from_state = initial_state
    t_exe = None
    snode_schedule_all = []
    for sweep_idx in range(sweep_num):
        gcheck.put_banned = [track_list[sweep_idx][2]]
        sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
    #     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
        goal_nodes = [("track_face",)*obj_num+sweep_goal]
        if sweep_idx < sweep_num-1:
            for i_s in range(obj_num):
                obj_goal = ["track_face"]*obj_num
                obj_goal[i_s] = "grip1"
                goal_nodes += [tuple(obj_goal)+sweep_goal]
        gtimer.tic("plan{}".format(sweep_idx))
        ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                      timeout_loop=20, multiprocess=True, timeout=5,
                      plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
        gtimer.toc("plan{}".format(sweep_idx))
        schedules = ppline.tplan.find_schedules()
        schedules_sorted = ppline.tplan.sort_schedule(schedules)
        snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
        snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
        snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
        snode_schedule = mix_schedule(mplan, snode_schedule_safe)
        from_state = snode_schedule[-1].state
        if t_exe:
            t_exe.join()
        else:
            gtimer.toc("firstmove")
        t_exe = Thread(target=ppline.execute_schedule, args = (snode_schedule,), kwargs=dict(auto_stop=False))
        t_exe.start()
        snode_schedule_all.append(snode_schedule)
    t_exe.join()
    crob.stop_tracking()

In [37]:
print(gtimer)

detect: 	175.0 ms/1 = 174.81 ms (174.81/174.81)
firstmove: 	4552.0 ms/1 = 4551.953 ms (4551.953/4551.953)
plan0: 	3496.0 ms/1 = 3495.868 ms (3495.868/3495.868)
initialize_memory: 	22.0 ms/3 = 7.271 ms (6.438/8.883)
init_search: 	43.0 ms/3 = 14.271 ms (10.722/19.792)
start_process: 	1842.0 ms/3 = 613.957 ms (550.054/669.434)
plan1: 	3133.0 ms/1 = 3132.877 ms (3132.877/3132.877)
plan2: 	5273.0 ms/1 = 5273.061 ms (5273.061/5273.061)

